In [21]:
import pandas as pd
!pwd
dataset = pd.read_csv('/Users/nikhil/pre_processed_patient_diagnostics.csv', encoding='ISO-8859-1');

dataset.columns

/Users/nikhil


Index(['Unnamed: 0', 'Case', 'Age', 'Text Details', 'Surgery - Yes OR No',
       'word_count'],
      dtype='object')

In [22]:
dataset['Surgery - Yes OR No'] = dataset['Surgery - Yes OR No'].astype(str) # taking the target column in seperate variable. we will add this to final data frame when pre processing is done
data_desc = dataset["Text Details"] 

In [23]:
for i in range(1,len(data_desc)):
    data_desc[i] = re.sub(" \d+", " ",data_desc[i]) # this will retain only the words

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
data_mat = bow.fit_transform(data_desc)
dense = data_mat.todense()

In [25]:
# extracting the features and storing it in a different dataset
featureset = pd.DataFrame(dense,columns=bow.get_feature_names())
featureset["Surgery - Yes OR No"] = dataset['Surgery - Yes OR No']
classlabel = "Surgery - Yes OR No"


print(featureset.shape)

(117, 1001)


In [27]:
# shuffing the data to avoid baising 
import numpy as np
featureset = featureset.sample(frac = 1)
print(featureset.shape)
Targetindex = featureset.columns.get_loc("Surgery - Yes OR No")
np.random.seed(0)

(117, 1001)


In [28]:
def Average(lst):
    return sum(lst) / len(lst)

In [30]:
#Test split out of training (Divided training into test and train sets respectively)
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
def splitter(trials,data,size):
    
    model_acc=[]
    model_f1=[]
    model_recall=[]
    model_precision=[]
    
  
    for i in range(0,trials):
        dataset_train, dataset_test = train_test_split(data, test_size=size)
        dataset_train = pd.DataFrame(dataset_train)
        dataset_test = pd.DataFrame(dataset_test)
        dataset_train.columns = featureset.columns
        dataset_test.columns = featureset.columns
        Targetindex = dataset_train.columns.get_loc("Surgery - Yes OR No")
        
        X = dataset_train.drop(dataset_train.columns[[Targetindex]], axis=1) # all the features except the target variable
        y = dataset_train["Surgery - Yes OR No"] # only the target variable
        X1 = dataset_test.drop(dataset_test.columns[[Targetindex]], axis=1) # all the features except the target variable
        y1 = dataset_test["Surgery - Yes OR No"] # only the target variable
        
        # label encode the target variable 
        encoder = preprocessing.LabelEncoder()
        train_y = encoder.fit_transform(y)
        valid_y = encoder.fit_transform(y1)
        
        clf = linear_model.LogisticRegression().fit(X, train_y)
        
        testpred = clf.predict(X1)
        
        model_acc.append(accuracy_score(valid_y,testpred))
        model_f1.append(f1_score(valid_y,testpred,pos_label=1,average="binary"))
        model_recall.append(recall_score(valid_y,testpred,pos_label=1,average="binary"))
        model_precision.append(precision_score(valid_y,testpred,pos_label=1,average="binary"))
        
        matrix=confusion_matrix(valid_y,testpred)
        print("Confusion matrix for split",i)
        print(matrix)
        
    print("Average metrics for Logistic Regression are:")
    
    acc_avg=Average(model_acc)
    f1_avg=Average(model_f1)
    recall_avg=Average(model_recall)
    precision_avg=Average(model_precision)

    return acc_avg,f1_avg,recall_avg,precision_avg




In [34]:
splitter(10,np.array(featureset),0.3)

Confusion matrix for split 0
[[ 3  1]
 [ 0 32]]
Confusion matrix for split 1
[[ 2  6]
 [ 0 28]]
Confusion matrix for split 2
[[ 1  3]
 [ 0 32]]
Confusion matrix for split 3
[[ 3  2]
 [ 1 30]]
Confusion matrix for split 4
[[ 5  0]
 [ 0 31]]
Confusion matrix for split 5
[[ 3  4]
 [ 2 27]]
Confusion matrix for split 6
[[ 0  7]
 [ 0 29]]
Confusion matrix for split 7
[[ 1  3]
 [ 1 31]]
Confusion matrix for split 8
[[ 2  7]
 [ 1 26]]
Confusion matrix for split 9
[[ 2  6]
 [ 0 28]]
Average metrics for Logistic Regression are:


(0.8777777777777777,
 0.9297040128864877,
 0.9830489381205455,
 0.8844708298764277)